In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss

In [2]:
tf.random.set_seed(42)

In [ ]:
raw_data = pd.read_csv('All_tnj_rain_gauge_daily.csv')
raw_data.head()

In [ ]:
raw_data.dtypes

In [ ]:
raw_data.isnull().sum()

In [6]:
raw_data.fillna(raw_data.mean(),inplace=True)

In [ ]:
raw_data.isnull().sum()

In [8]:
date = []
for i in range(len(raw_data['YEAR'])):
  date.append(str(raw_data['YEAR'][i])+'/'+str(raw_data['MO'][i])+'/'+str(raw_data['DY'][i]))

In [9]:
df = pd.DataFrame()
df = raw_data.drop(['YEAR','MO','DY'],axis=1)
df['DATE'] = pd.to_datetime(date)

In [ ]:
df2 =  df.copy()

df2['DATE'] = pd.to_datetime(df2['DATE'])

df2.set_index('DATE', inplace=True)

alpha = 0.05


sample_size = 1000


sampled_data = df2.sample(n=sample_size, random_state=42)


kpss_result = kpss(sampled_data['PRECTOTCORR'])
kpss_statistic, kpss_pvalue, _, kpss_crit_values = kpss_result
print("Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test (Sampled Data):")
print(f"KPSS Statistic: {kpss_statistic}")
print(f"P-value: {kpss_pvalue}")
print("Critical Values:")
for key, value in kpss_crit_values.items():
    print(f"{key}: {value}")
if kpss_pvalue <= alpha:
    print("Result: Reject null hypothesis, trend is not present (stationary).")
else:
    print("Result: Fail to reject null hypothesis, trend is present (non-stationary).")


adf_result = adfuller(sampled_data['PRECTOTCORR'])
adf_statistic = adf_result[0]
adf_critical_values = adf_result[4]

print("Augmented Dickey-Fuller (ADF) Test (Sampled Data):")
print(f"ADF Statistic: {adf_statistic}")

if adf_statistic < adf_critical_values['5%']:
    print("Result: Reject null hypothesis, trend is present (non-stationary).")
else:
    print("Result: Fail to reject null hypothesis, trend is not present (stationary).")


In [ ]:
df1 = df.copy()

df1['DATE'] = pd.to_datetime(df1['DATE'])


df1.set_index('DATE', inplace=True)


df1 = df1.resample('M').sum()


result = sm.tsa.seasonal_decompose(df1['PRECTOTCORR'], model='additive')

plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(df1['PRECTOTCORR'], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(result.trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(result.seasonal, label='Seasonal')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(result.resid, label='Residual')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [12]:
selected_columns = ['T2M', 'T2MDEW', 'T2M_MAX', 'T2M_MIN', 'RH2M', 'PS', 'WS10M', 'PRECTOTCORR']
data = df[selected_columns].values

scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [13]:
train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

In [14]:
def create_sequences(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), :-1])
        y.append(data[i + look_back, -1])
    return np.array(X), np.array(y)

In [15]:
look_back = 10
trainX, trainY = create_sequences(train_data, look_back)
testX, testY = create_sequences(test_data, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))

In [16]:
def objective_function(hyperparameters):
    gru_units = hyperparameters['gru_units']
    optimizer = 'adam'
    epochs = hyperparameters['epochs']
    batch_size = hyperparameters['batch_size']

    # Create and compile the LSTM model with the given hyperparameters
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.GRU(gru_units, input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=0)

    # Make predictions on the test set
    y_pred = model.predict(testX)

    # Calculate the RMSE as the evaluation metric
    rmse = np.sqrt(mean_squared_error(testY, y_pred))
    print(rmse)

    return rmse

In [ ]:
population_size = 6
max_iterations = 2

# Define the search space for hyperparameters
search_space = {
    'gru_units': [32, 64, 128],
    'epochs': [40, 50],
    'batch_size': [32, 64],
    # Add more hyperparameters and their ranges here
}

# Initialize the population with random hyperparameters
population = [{'gru_units': np.random.choice(search_space['gru_units']),
              'epochs': np.random.choice(search_space['epochs']),
              'batch_size': np.random.choice(search_space['batch_size'])} for _ in range(population_size)]

for iteration in range(max_iterations):
    # Calculate fitness values for each set of hyperparameters in the population
    fitness = [objective_function(hyperparameters) for hyperparameters in population]

    # Find the best and worst solutions in the population
    best_index = np.argmin(fitness)
    worst_index = np.argmax(fitness)

    # Update the position of the Harris hawks (i.e., update hyperparameters)
    for i in range(population_size):
        if i != best_index:
            E0 = np.random.rand()  # Random exploration factor
            E1 = 2 * np.random.rand() - 1  # Random exploration factor
            E2 = 2 * np.random.rand() - 1  # Random exploration factor
            E3 = 2 * np.random.rand() - 1  # Random exploration factor

            if fitness[i] > fitness[best_index]:
                for param in search_space.keys():
                    population[i][param] = int(population[i][param]) + int(E0 * (int(population[best_index][param]) - int(population[i][param])))
            if fitness[i] < fitness[best_index]:
                for param in search_space.keys():
                    population[i][param] = int(population[i][param]) + int(E1 * (int(population[i][param]) - int(population[best_index][param])))

            if i != worst_index:
              distance = np.linalg.norm(np.array(list(population[worst_index].values())) - np.array(list(population[i].values())))
              if distance > 1e-6:
                for param in search_space.keys():
                  if isinstance(population[i][param], (int, float)):
                    population[i][param] = int(population[i][param]) + int(E2 * (population[worst_index][param] - population[i][param]) / distance)
                  else:
                    population[i][param] = np.array(list(population[i][param])) + E2 * (np.array(list(population[worst_index].values())) - np.array(list(population[i].values()))) / distance
              else:
                for param in search_space.keys():
                  if isinstance(population[i][param], (int, float)):
                    population[i][param] = int(population[i][param]) + int(E3 * np.random.rand())
                  else:
                    population[i][param] = np.array(list(population[i][param])) + E3 * np.random.rand()




            # Ensure hyperparameters stay within bounds
            for param in search_space.keys():
              if isinstance(population[i][param], (int, float)):
                population[i][param] = int(np.clip(population[i][param], min(search_space[param]), max(search_space[param])))
              else:
                population[i][param] = np.clip(population[i][param], min(search_space[param]), max(search_space[param]))

# Find the best set of hyperparameters and its corresponding fitness value
best_hyperparameters = population[best_index]
best_fitness = fitness[best_index]

print("Best Hyperparameters:", best_hyperparameters)
print("Best Fitness Value (RMSE):", best_fitness)


In [ ]:
#units = best_hyperparameters['gru_units']
#epoch = best_hyperparameters['epochs']
#batch = best_hyperparameters['batch_size']
model_GRU = tf.keras.Sequential()
model_GRU.add(tf.keras.layers.GRU(128, input_shape=(look_back,7)))
model_GRU.add(tf.keras.layers.Dense(1))
model_GRU.compile(loss='mean_squared_error', optimizer='adam')
history_GRU = model_GRU.fit(trainX, trainY,validation_data=(testX,testY), epochs=40, batch_size=50)

In [ ]:
trainPredict = model_GRU.predict(trainX)
testPredict = model_GRU.predict(testX)

In [ ]:
trainScore = np.sqrt(mean_squared_error(trainY, trainPredict))
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Train RMSE: {trainScore:.2f}')
print(f'Test RMSE: {testScore:.2f}')

In [ ]:
plt.figure()
plt.plot(history_GRU.history['loss'],label="LOSS")
plt.plot(history_GRU.history['val_loss'],label = "VALIDATION LOSS")
plt.legend()
plt.show()

In [ ]:
plt.plot(testY)
plt.plot(testPredict)

In [ ]:
mse = mean_squared_error(testY,testPredict)
rmse = np.sqrt(mse)
mae = mean_absolute_error(testY,testPredict)
r2 = r2_score(testY,testPredict)
mbd = np.mean(testPredict - testY)
print(f"mean squared error: {mse}")
print(f"root mean squared error: {rmse}")
print(f"mean absolute error: {mae}")
print(f"mean bias deviation {mbd}")
print(f"r2_score: {r2}")

In [ ]:
df = pd.DataFrame()
df['Model'] = 'LSTM'
df['MSE'] = mse
df['MAE'] =  mae
df['MBD'] = mbd
df['R2'] = r2

df.to_csv("GRU.csv",index=False)